### Split Array Largest Sum

Given an array which consists of non-negative integers and an integer m, you can split the array into m non-empty continuous subarrays. Write an algorithm to minimize the largest sum among these m subarrays.

```
Note:
If n is the length of array, assume the following constraints are satisfied:

1 ≤ n ≤ 1000
1 ≤ m ≤ min(50, n)
Examples:

Input:
nums = [7,2,5,10,8]
m = 2

Output:
18

Explanation:
There are four ways to split nums into two subarrays.
The best way is to split it into [7,2,5] and [10,8],
where the largest sum among the two subarrays is only 18.
```

In [ ]:
# dp o(mn^2)
class Solution: 
    def splitArray(self, nums: List[int], m: int) -> int:
        n = len(nums)
        cums = [0]
        for i in range(n):
            cums.append(cums[i]+nums[i])
        
        dp = [[float('inf') for j in range(n+1)]
                            for i in range(m+1)]
        # stands for m cut in nums[:n]
        dp[0][0] = 0 
        #O(mn^2)
        for i in range(1, m+1):
            for j in range(1, n+1):
                for k in range(i-1, j):
                    # val = max(dp[i-1][k], sum(nums[k:j]))
                    # dp[i-1][k]表示数组中前k个数字分成i-1组所能得到的最小的各个子数组中最大值
                    val = max(dp[i-1][k], cums[j]-cums[k])
                    dp[i][j] = min(dp[i][j], val)
        
        return dp[m][n]

In [5]:
#O(n*32)
class Solution:
    def splitArray(self, nums: list, m: int) -> int:
        
        def check(mid):#O(n)
            # can we make at-most m sub arrays with maxium sum at most mid
            cuts, cur_sum = 0, 0 
            for x in nums:
                cur_sum += x 
                if cur_sum > mid:
                    cuts, cur_sum = cuts+1, x
            subs = cuts + 1 
            return (subs <= m)
        
        #O(n)
        start, end = max(nums), sum(nums)
        
        #O(32) if int is bounded to 2^32
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if check(mid):
                end = mid
            else:
                start = mid 
        
        if check(start):
            return start
        return end

### Odd Even Jump(Monotone Stack)

You are given an integer array A.  From some starting index, you can make a series of jumps.  The (1st, 3rd, 5th, ...) jumps in the series are called odd numbered jumps, and the (2nd, 4th, 6th, ...) jumps in the series are called even numbered jumps.

You may from index i jump forward to index j (with i < j) in the following way:

During odd numbered jumps (ie. jumps 1, 3, 5, ...), you jump to the index j such that A[i] <= A[j] and A[j] is the smallest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
During even numbered jumps (ie. jumps 2, 4, 6, ...), you jump to the index j such that A[i] >= A[j] and A[j] is the largest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
(It may be the case that for some index i, there are no legal jumps.)
A starting index is good if, starting from that index, you can reach the end of the array (index A.length - 1) by jumping some number of times (possibly 0 or more than once.)

Return the number of good starting indexes.

```
Example 1:

Input: [10,13,12,14,15]
Output: 2
Explanation: 
From starting index i = 0, we can jump to i = 2 (since A[2] is the smallest among A[1], A[2], A[3], A[4] that is greater or equal to A[0]), then we can't jump any more.
From starting index i = 1 and i = 2, we can jump to i = 3, then we can't jump any more.
From starting index i = 3, we can jump to i = 4, so we've reached the end.
From starting index i = 4, we've reached the end already.
In total, there are 2 different starting indexes (i = 3, i = 4) where we can reach the end with some number of jumps.
```

In [ ]:
class Solution:
    def oddEvenJumps(self, A: List[int]) -> int:
        n = len(A)
        next_higher, next_lower = [0]*n, [0]*n
        
        stack = []
        
        for a, i in sorted([a,i] for i, a in enumerate(A)):
            while stack and stack[-1] < i:
                next_higher[stack.pop()] = i
            stack.append(i)
            
        stack = []
        
        for a, i in sorted([-a,i] for i, a in enumerate(A)):
            while stack and stack[-1] < i:
                next_lower[stack.pop()] = i 
            stack.append(i)
        
        higher, lower = [0]*n, [0]*n
        higher[-1] = lower[-1] = 1 
        
        for i in range(n-1)[::-1]:
            higher[i] = lower[next_higher[i]]
            lower[i] = higher[next_lower[i]]
        
        return sum(higher)

### 单调栈小结

所谓的单调栈 Monotone Stack，就是栈内元素都是单调递增或者单调递减的，有时候需要严格的单调递增或递减，根据题目的具体情况来看吧。举个生动的例子来说明吧：比如有一天，某家店在发 free food，很多人在排队，于是你也赶过去凑热闹。但是由于来晚了，队伍已经很长了，想着不然就插个队啥的。但发现排在队伍最前面的都是一些有纹身的大佬，惹不起，只能赞美道，小猪佩奇身上纹，来世还做社会人。于是往队伍后面走，发现是一群小屁孩，直接全部撵走，然后排在了社会大佬们的后面。那么这就是一个单调递减的栈，按实力递减。由于栈元素是后进先出的，所以上面的例子正确的检查顺序应该是从队尾往前遍历，小屁孩都撵走，直到遇到大佬停止，然后排在大佬后面（假设这个队列已经事先按实力递减排好了）。

1.单调栈里的元素具有单调性

2.元素加入栈前，会在栈顶端把破坏栈单调性的元素都删除

3.使用单调栈（单增）可以找到元素向左遍历第一个比他小的元素，也可以找到元素向左遍历第一个比他大的元素。

### Next Exceed

给一个数组，返回一个大小相同的数组。返回的数组的第i个位置的值应当是，对于原数组中的第i个元素，至少往右走多少步，才能遇到一个比自己大的元素（如果之后没有比自己大的元素，或者已经是最后一个元素，则在返回数组的对应位置放上-1）。

简单的例子：

input: 5,3,1,2,4

return: -1 3 1 1 -1

In [3]:
def nextExceed(A:list):
    n = len(A)
    res = [-1] * n
    stack = []
    for i in range(n):
        while stack and A[i] > A[stack[-1]]:
        #前面比我弱的都踢了，
            res[stack[-1]] = i - stack[-1]
            stack.pop()
        stack.append(i)
    print(res)

nextExceed([5,3,1,2,4])            

[-1, 3, 1, 1, -1]


### Largest Rectangle in Histogram


Given n non-negative integers representing the histogram's bar height where the width of each bar is 1, find the area of largest rectangle in the histogram.

 
![](https://assets.leetcode.com/uploads/2018/10/12/histogram_area.png)

Above is a histogram where width of each bar is 1, given height = [2,1,5,6,2,3].

 


The largest rectangle is shown in the shaded area, which has area = 10 unit.

In [5]:
class Solution:
    def largestRectangleArea(self, heights: list) -> int:
        heights.append(0)
        n = len(heights)
        stack = [-1] 
        ans = 0 
        for i in range(n):
            while heights[i] < heights[stack[-1]]: 
                # 单增栈，前面比我大的都T了，我就是前面比我大的这些里面next smaller
                # 栈顶元素高度是矩形的高度
                h = heights[stack[-1]]
                stack.pop() # 栈顶是矩形位置，必须在这里pop，因为前面可能有gap
                ans = max(ans, h * (i - stack[-1] - 1))        
            stack.append(i)
        heights.pop()
        return ans
Solution().largestRectangleArea([4, 1, 3, 2, 2])

6

### Maximal Rectangle
Given a 2D binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.

```
Example:

Input:
[
  ["1","0","1","0","0"],
  ["1","0","1","1","1"],
  ["1","1","1","1","1"],
  ["1","0","0","1","0"]
]
Output: 6
```

In [ ]:
class Solution:
    def maximalRectangle(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        ans, heights = 0, [0] * len(matrix[0])
        for row in matrix:
            for index, num in enumerate(row):
                heights[index] = heights[index] + 1 if num == '1' else 0
            
            ans = max(ans, self.maxRect(heights))
        return ans      
          
    def maxRect(self, A:list):
        A.append(0)
        stack = [-1]
        ans = 0
        for i in range(len(A)):
            while A[i] < A[stack[-1]]:
                h = A[stack.pop()]
                ans = max(ans, h*(i - stack[-1] - 1))
            stack.append(i)
        A.pop()
        return ans
                

### Trapping Rain Water


Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining.

![](https://assets.leetcode.com/uploads/2018/10/22/rainwatertrap.png)

The above elevation map is represented by array [0,1,0,2,1,0,1,3,2,1,2,1]. In this case, 6 units of rain water (blue section) are being trapped. Thanks Marcos for contributing this image!

Example:

Input: [0,1,0,2,1,0,1,3,2,1,2,1]

Output: 6

In [ ]:
class Solution:
    def maximalRectangle(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        ans, heights = 0, [0] * len(matrix[0])
        for row in matrix:
            for index, num in enumerate(row):
                heights[index] = heights[index] + 1 if num == '1' else 0
            ans = max(ans, self.maxRect(heights))
        return ans      
          
    def maxRect(self, A:list):
        A.append(0)
        stack = [-1]
        ans = 0
        for i in range(len(A)):
            while A[i] < A[stack[-1]]:
                h = A[stack.pop()]
                ans = max(ans, h*(i - stack[-1] - 1))
            stack.append(i)
        A.pop()
        return ans

### Campus Bikes
Medium
On a campus represented as a 2D grid, there are N workers and M bikes, with N <= M. Each worker and bike is a 2D coordinate on this grid.

Our goal is to assign a bike to each worker. Among the available bikes and workers, we choose the (worker, bike) pair with the shortest Manhattan distance between each other, and assign the bike to that worker. (If there are multiple (worker, bike) pairs with the same shortest Manhattan distance, we choose the pair with the smallest worker index; if there are multiple ways to do that, we choose the pair with the smallest bike index). We repeat this process until there are no available workers.

The Manhattan distance between two points p1 and p2 is Manhattan(p1, p2) = |p1.x - p2.x| + |p1.y - p2.y|.

Return a vector ans of length N, where ans[i] is the index (0-indexed) of the bike that the i-th worker is assigned to.

```
Input: workers = [[0,0],[2,1]], bikes = [[1,2],[3,3]]
Output: [1,0]
Explanation: 
Worker 1 grabs Bike 0 as they are closest (without ties), and Worker 0 is assigned Bike 1. So the output is [1, 0].
```

In [ ]:
class Solution:
    def assignBikes(self, workers: List[List[int]], bikes: List[List[int]]) -> List[int]:
        ans = [-1] * len(workers)
        used = set()
        dists = []
        
        for i in range(len(workers)):
            for j in range(len(bikes)):
                dis = abs(workers[i][0] - bikes[j][0]) + abs(workers[i][1] - bikes[j][1])
                dists.append((dis, i, j))      
        
        
        for dis, w, b in sorted(dists):
            if ans[w] == -1 and b not in used:
                ans[w] = b
                used.add(b)
        return ans

###  Count Complete Tree Nodes

Given a complete binary tree, count the number of nodes.

Note:

Definition of a complete binary tree from Wikipedia:
In a complete binary tree every level, except possibly the last, is completely filled, and all nodes in the last level are as far left as possible. It can have between 1 and 2h nodes inclusive at the last level h.
```
Example:

Input: 
    1
   / \
  2   3
 / \  /
4  5 6

Output: 6
```

In [ ]:
 class Solution:#O(lognlogn)
        def countNodes(self, root):
            if not root:
                return 0
            leftDepth = self.getDepth(root.left)
            rightDepth = self.getDepth(root.right)
            if leftDepth == rightDepth:
                return pow(2, leftDepth) + self.countNodes(root.right)
            else:
                return pow(2, rightDepth) + self.countNodes(root.left)
    
        def getDepth(self, root):
            if not root:
                return 0
            return 1 + self.getDepth(root.left)

### Guess the Word

This problem is an interactive problem new to the LeetCode platform.

We are given a word list of unique words, each word is 6 letters long, and one word in this list is chosen as secret.

You may call master.guess(word) to guess a word.  The guessed word should have type string and must be from the original list with 6 lowercase letters.

This function returns an integer type, representing the number of exact matches (value and position) of your guess to the secret word.  Also, if your guess is not in the given wordlist, it will return -1 instead.

For each test case, you have 10 guesses to guess the word. At the end of any number of calls, if you have made 10 or less calls to master.guess and at least one of these guesses was the secret, you pass the testcase.

Besides the example test case below, there will be 5 additional test cases, each with 100 words in the word list.  The letters of each word in those testcases were chosen independently at random from 'a' to 'z', such that every word in the given word lists is unique.
```
Example 1:
Input: secret = "acckzz", wordlist = ["acckzz","ccbazz","eiowzz","abcczz"]

Explanation:

master.guess("aaaaaa") returns -1, because "aaaaaa" is not in wordlist.
master.guess("acckzz") returns 6, because "acckzz" is secret and has all 6 matches.
master.guess("ccbazz") returns 3, because "ccbazz" has 3 matches.
master.guess("eiowzz") returns 2, because "eiowzz" has 2 matches.
master.guess("abcczz") returns 4, because "abcczz" has 4 matches.

We made 5 calls to master.guess and one of them was the secret, so we pass the test case.
```

In [ ]:
class Solution(object):
    def findSecretWord(self, wordlist, master):
        def pair_matches(a, b):         # count the number of matching characters
            return sum(c1 == c2 for c1, c2 in zip(a, b))

        def most_overlap_word():
            counts = [[0 for _ in range(26)] for _ in range(6)]     # counts[i][j] is nb of words with char j at index i
            for word in candidates:
                for i, c in enumerate(word):
                    counts[i][ord(c) - ord("a")] += 1

            best_score = 0
            for word in candidates:
                score = 0
                for i, c in enumerate(word):
                    score += counts[i][ord(c) - ord("a")]           # all words with same chars in same positions
                if score > best_score:
                    best_score = score
                    best_word = word

            return best_word

        candidates = wordlist[:]        # all remaining candidates, initially all words
        while candidates:

            s = most_overlap_word()     # guess the word that overlaps with most others
            matches = master.guess(s)

            if matches == 6:
                return

            candidates = [w for w in candidates if pair_matches(s, w) == matches]   # filter words with same matches